In [1]:
import pandas as pd
import librosa
import os
import numpy as np 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

In [2]:
from keras import backend as keras_backend
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, LeakyReLU, SpatialDropout2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from keras.callbacks import ModelCheckpoint 
from keras.regularizers import l2

In [7]:
import torch
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

ModuleNotFoundError: No module named 'torch'

In [8]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [1]:
import tensorflow as  tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


ModuleNotFoundError: No module named 'tensorflow'

## Arquivo de metadados

In [3]:
audio_path = 'spectogram/'
spectograms = os.listdir(audio_path)

#metadata with most_freq chords indicator
df = pd.read_csv('metadata_final.csv', sep = ',')
df_chords = df[df['most_freq'] == 1]


## Mel Spectogram

In [4]:
X = np.load("X-mel_spec.npy")
y = np.load("y-mel_spec.npy")

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [41]:
le = LabelEncoder()
y_test_encoded = to_categorical(le.fit_transform(y_test))
y_train_encoded = to_categorical(le.fit_transform(y_train))


mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [7]:
X_train.shape

(755, 40, 608)

In [11]:
import mlflow

mlflow.tensorflow.autolog(every_n_iter = 2)

2023/10/21 14:30:24 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


In [12]:
# How data should be structured
num_rows = 608
num_columns = 40 
num_channels = 1

# Reshape to fit the network input (channel last)
X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

# Total number of labels to predict (equal to the network output nodes)
num_labels = y_train_encoded.shape[1]

In [13]:
def create_model():

    # Create a secquential object
    model = Sequential()


    # Conv 1
    model.add(Conv2D(filters=32, 
                     kernel_size=(3, 3), 
                     input_shape=(num_rows, num_columns, num_channels)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=32, 
                     kernel_size=(3, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())


    # Max Pooling #1
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, 
                     kernel_size=(3, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=64, 
                     kernel_size=(3,3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    
   
    # Reduces each h×w feature map to a single number by taking the average of all h,w values.
    model.add(GlobalAveragePooling2D())


    # Softmax output
    model.add(Dense(num_labels, activation='softmax'))
    
    return model


model = create_model()

In [14]:
model.compile(
    loss='binary_crossentropy',  # duas classes
    metrics=['accuracy'],  
    optimizer='adam')

# Display model architecture summary 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 606, 38, 32)       320       
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 606, 38, 32)       0         
                                                                 
 batch_normalization_4 (Bat  (None, 606, 38, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 604, 36, 32)       9248      
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 604, 36, 32)       0         
                                                                 
 batch_normalization_5 (Bat  (None, 604, 36, 32)       128       
 chNormalization)                                     

In [16]:
history = model.fit(X_train, 
                    y_train_encoded, 
                    batch_size=4, 
                    epochs=50, 
                    validation_data=(X_test,y_test_encoded),
                    verbose=1)

2023/10/21 14:36:50 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3f03783cb36b47a38c15da06eb491eb0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50
189/189 [==============================] - 51s 272ms/step - loss: 0.3140 - accuracy: 0.1881 - val_loss: 0.3110 - val_accuracy: 0.1957
Epoch 2/50
189/189 [==============================] - 46s 240ms/step - loss: 0.3124 - accuracy: 0.1921 - val_loss: 0.3127 - val_accuracy: 0.1903
Epoch 3/50
189/189 [==============================] - 45s 239ms/step - loss: 0.3095 - accuracy: 0.2079 - val_loss: 0.3158 - val_accuracy: 0.1769
Epoch 4/50
189/189 [==============================] - 46s 242ms/step - loss: 0.3055 - accuracy: 0.2238 - val_loss: 0.3106 - val_accuracy: 0.2413
Epoch 5/50
189/189 [==============================] - 46s 242ms/step - loss: 0.3052 - accuracy: 0.2212 - val_loss: 0.3039 - val_accuracy: 0.2386
Epoch 6/50
189/189 [==============================] - 49s 261ms/step - loss: 0.3017 - accuracy: 0.2543 - val_loss: 0.3076 - val_accuracy: 0.1957
Epoch 7/50
189/189 [==============================] - 50s 264ms/step - loss: 0.2990 - accuracy: 0.2344 - val_loss: 0.3078 - val_ac

INFO:tensorflow:Assets written to: C:\Users\alves\AppData\Local\Temp\tmpwchs0yyz\model\data\model\assets


## Metrics

In [54]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

preds_score = model.predict(X_test)
pred_idx = tf.math.argmax(preds_score, axis = 1)
y_test_idx = tf.math.argmax(y_test_encoded, axis = 1)

target_names = mapping.keys()

print('F1-Score: ', f1_score(y_test_idx, pred_idx, average = 'macro'))

print(classification_report(y_test_idx, pred_idx, target_names= target_names))


12/12 [==============================] - 4s 328ms/step
F1-Score:  0.3002048912124947
              precision    recall  f1-score   support

      A#:maj       0.33      0.04      0.07        27
      C#:maj       0.50      0.15      0.24        39
       C:maj       0.33      0.03      0.05        37
      D#:maj       0.30      0.78      0.43        27
       D:maj       0.29      0.70      0.41        37
       E:maj       0.52      0.62      0.57        37
      F#:maj       0.31      0.70      0.43        33
       F:maj       0.39      0.34      0.36        50
      G#:maj       0.75      0.15      0.26        39
       G:maj       0.30      0.15      0.20        47

    accuracy                           0.35       373
   macro avg       0.40      0.37      0.30       373
weighted avg       0.41      0.35      0.30       373



In [35]:
from sklearn.metrics import f1_score
import tensorflow as tf
#print('Acurácia: ', f1_score(y_test, preds))

<tf.Tensor: shape=(373,), dtype=int64, numpy=
array([4, 1, 2, 1, 0, 7, 1, 3, 6, 4, 6, 4, 6, 7, 5, 8, 5, 8, 3, 8, 5, 0,
       0, 7, 7, 1, 3, 3, 7, 2, 5, 7, 3, 2, 9, 5, 4, 4, 3, 4, 3, 9, 0, 6,
       5, 8, 9, 1, 3, 4, 7, 8, 8, 1, 5, 4, 7, 1, 7, 7, 4, 9, 9, 0, 9, 7,
       8, 9, 9, 3, 1, 1, 7, 9, 0, 7, 7, 1, 8, 2, 8, 9, 7, 5, 1, 0, 7, 5,
       1, 1, 5, 4, 2, 8, 0, 7, 9, 5, 1, 6, 9, 5, 8, 9, 4, 8, 2, 6, 9, 0,
       9, 2, 5, 4, 7, 7, 3, 6, 0, 9, 6, 5, 6, 3, 1, 5, 3, 1, 2, 4, 7, 5,
       9, 2, 8, 2, 4, 6, 2, 6, 9, 4, 3, 1, 4, 9, 1, 4, 8, 9, 5, 6, 6, 7,
       7, 1, 3, 7, 3, 8, 2, 7, 1, 6, 5, 9, 4, 2, 8, 6, 9, 2, 2, 6, 0, 5,
       2, 8, 0, 8, 8, 0, 5, 0, 7, 4, 7, 5, 1, 9, 2, 9, 7, 8, 0, 4, 7, 3,
       3, 7, 0, 9, 6, 2, 9, 3, 0, 9, 9, 6, 4, 1, 3, 2, 7, 4, 4, 9, 8, 6,
       1, 7, 1, 5, 0, 1, 8, 2, 2, 6, 7, 1, 7, 7, 6, 1, 2, 4, 1, 7, 3, 1,
       8, 4, 9, 4, 9, 7, 8, 5, 8, 4, 9, 2, 9, 2, 8, 7, 1, 4, 1, 4, 2, 1,
       0, 0, 1, 8, 2, 2, 6, 6, 4, 9, 7, 4, 7, 5, 3, 4, 5, 7, 9, 5, 5, 9,
     